# CAD integration

In this task, we will learn how to run CAD-based simulations.

Our example case will be a 3D ITER-like monoblock made of three different materials (tungsten, cucrzr, and copper).

For this example, the mesh was created with [SALOME](https://www.salome-platform.org/) but other meshing software can be used like [GMSH](https://gmsh.info/).

Using [meshio](https://github.com/nschloe/meshio), the mesh file (here a .med file) can be converted to .xdmf format, readable by FESTIM.

We wrote the following function `convert_med_to_xdmf` to convert .med files to .xdmf.

In [1]:
import meshio


def convert_med_to_xdmf(
    med_file,
    cell_file="mesh_domains.xdmf",
    facet_file="mesh_boundaries.xdmf",
    cell_type="tetra",
    facet_type="triangle",
):
    """Converts a MED mesh to XDMF
    Args:
        med_file (str): the name of the MED file
        cell_file (str, optional): the name of the file containing the
            volume markers. Defaults to "mesh_domains.xdmf".
        facet_file (str, optional): the name of the file containing the
            surface markers.. Defaults to "mesh_boundaries.xdmf".
        cell_type (str, optional): The topology of the cells. Defaults to "tetra".
        facet_type (str, optional): The topology of the facets. Defaults to "triangle".
    Returns:
        dict, dict: the correspondance dict, the cell types
    """
    msh = meshio.read(med_file)

    correspondance_dict = msh.cell_tags

    cell_data_types = msh.cell_data_dict["cell_tags"].keys()

    for mesh_block in msh.cells:
        if mesh_block.type == cell_type:

            meshio.write_points_cells(
                cell_file,
                msh.points,
                [mesh_block],
                cell_data={"f": [-1 * msh.cell_data_dict["cell_tags"][cell_type]]},
            )
        elif mesh_block.type == facet_type:
            meshio.write_points_cells(
                facet_file,
                msh.points,
                [mesh_block],
                cell_data={"f": [-1 * msh.cell_data_dict["cell_tags"][facet_type]]},
            )

    return correspondance_dict, cell_data_types

The subdomains (volumes and surfaces) in the .med file were given tags.

This is very important to be able to assign materials properties and boundary conditions to these subdomains.

The correspondance between tags and subdomains is returned by `convert_med_to_xdmf`.
Here for example, the volume corresponding to `tungsten` is the tag `6`.

In [2]:
correspondance_dict, cell_data_types = convert_med_to_xdmf("task7/mesh.med", cell_file="task7/mesh_domains.xdmf", facet_file="task7/mesh_boundaries.xdmf")

print(correspondance_dict)

{-6: ['tungsten'], -7: ['cu'], -8: ['cucrzr'], -9: ['top_surface'], -10: ['cooling_surface'], -11: ['poloidal_gap'], -12: ['toroidal_gap'], -13: ['bottom']}


The converted .xdmf files can then be imported in FESTIM using the `MeshFromXDMF` class:

In [3]:
import festim as F

my_model = F.Simulation()

my_model.mesh = F.MeshFromXDMF(volume_file="task7/mesh_domains.xdmf", boundary_file="task7/mesh_boundaries.xdmf")

Succesfully load mesh with 106966 cells


Using the tags provided by `correspondance_dict`, we can create materials and assign them to the simulation:

In [4]:
tungsten = F.Material(
    id=6,
    D_0=4.1e-7,
    E_D=0.39,
    S_0=1.87e24,
    E_S=1.04,
    thermal_cond=100,
)

copper = F.Material(
    id=7,
    D_0=6.6e-7,
    E_D=0.387,
    S_0=3.14e24,
    E_S=0.572,
    thermal_cond=350,
)

cucrzr = F.Material(
    id=8,
    D_0=3.92e-7,
    E_D=0.418,
    S_0=4.28e23,
    E_S=0.387,
    thermal_cond=350
)

my_model.materials = [tungsten, copper, cucrzr]

Similarily, the surface tags are used to create boundary conditions:

In [5]:
heat_flux_top = F.FluxBC(surfaces=9, value=10e6, field="T")
convective_flux_coolant = F.ConvectiveFlux(surfaces=10, h_coeff=7e04, T_ext=323)
implantation_flux_top = F.ImplantationDirichlet(surfaces=9, phi=1.61e22, R_p=9.52e-10, D_0=tungsten.D_0, E_D=tungsten.E_D)
recombination_flux = F.DirichletBC(surfaces=[10, 11, 12, 13], value=0, field=0)

my_model.boundary_conditions = [heat_flux_top, convective_flux_coolant, implantation_flux_top, recombination_flux]

We will solve a steady state heat transfer problem:

In [6]:
my_model.T = F.HeatTransferProblem(transient=False)

Finally, we add the settings (with `chemical_pot=True` to account for conservation of chemical potential) and we export the temperature and mobile concentration to XDMF.

In [7]:
my_model.settings = F.Settings(
    absolute_tolerance=1e10,
    relative_tolerance=1e-10,
    transient=False,
    chemical_pot=True
)

my_model.exports = [F.XDMFExport("T"), F.XDMFExport("solute")]

my_model.initialise()
my_model.run()

Defining variational problem heat transfers
Solving stationary heat equation
Defining initial values
Defining variational problem
Defining source terms
Defining boundary conditions
Solving steady state problem...
Solved problem in 12.60 s


## Post processing

The recommended way of plotting 3D fields is to export them to XDMF using `XDMFExport`, download the .xdmf file and open it in [Paraview](https://www.paraview.org/).

This is what the fields look like in Paraview:

<img width="536" alt="image" src="https://github.com/pyvista/pyvista/assets/40028739/917a2d4f-96c0-4ad0-811f-7844e113c4aa">
<img width="520" alt="image" src="https://github.com/pyvista/pyvista/assets/40028739/324532f1-d5e9-42a5-8be4-8056da8c5f7c">

In [ ]:
import pyvista

# this is required to run on Codespaces
pyvista.global_theme.trame.server_proxy_enabled = True
pyvista.set_jupyter_backend('panel')

from dolfin_pyvista_adapter import plot

plot(my_model.T.T, show_edges=False, off_screen=False)

mobile_conc = my_model.h_transport_problem.mobile.post_processing_solution
plot(mobile_conc, show_edges=False, off_screen=False)